In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#import libraries
from sklearn.feature_selection import SelectKBest,chi2, f_classif ,mutual_info_classif ,f_regression ,SelectFdr ,mutual_info_regression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.legacy import Adam
import keras
# Conv1D + LSTM
from keras.layers import Conv1D 
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten

# **Reading Data**

In [4]:
df_dk=pd.read_csv('/Users/Abbas/Desktop/college/research work/dr_sheikh_abdullah/Keylogger_Detection.csv')
df_dk.head(5)

/var/folders/8z/5hrcm2_s3gbc_m2tqx33mw900000gp/T/ipykernel_74405/3046805388.py:1: DtypeWarning: Columns (48,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dk=pd.read_csv('/Users/Abbas/Desktop/college/research work/dr_sheikh_abdullah/Keylogger_Detection.csv')


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Class
0,0,10.42.0.211-52.6.25.230-34451-443-6,10.42.0.211,34451.0,52.6.25.230,443.0,6.0,04/08/2017 05:12:36,12140931.0,9.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,1,172.217.3.99-10.42.0.151-443-53892-6,10.42.0.151,53892.0,172.217.3.99,443.0,6.0,04/08/2017 07:55:51,418882.0,102.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,2,172.217.3.98-10.42.0.151-443-50750-6,172.217.3.98,443.0,10.42.0.151,50750.0,6.0,04/08/2017 08:48:19,45.0,2.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,3,10.42.0.211-10.42.0.1-23025-53-17,10.42.0.211,23025.0,10.42.0.1,53.0,17.0,04/08/2017 05:54:10,541699.0,1.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,4,10.42.0.211-123.129.244.226-52602-443-6,10.42.0.211,52602.0,123.129.244.226,443.0,6.0,04/08/2017 08:44:25,7310795.0,3.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


# **Data Preparation**

In [5]:
df_dk.dtypes

Unnamed: 0           int64
Flow ID             object
 Source IP          object
 Source Port       float64
 Destination IP     object
                    ...   
Idle Mean          float64
 Idle Std          float64
 Idle Max          float64
 Idle Min          float64
Class               object
Length: 86, dtype: object

In [6]:
Class_df = df_dk["Class"]
df_num = df_dk.select_dtypes(include=[np.number])
df_num = df_num.join(Class_df)

In [7]:
df_num.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
print(df_num.isna().sum().sum())
df_num=df_num.dropna()

879


In [9]:
df_num["label"]=df_num["Class"]
df_num.loc[df_num.label == "Benign", 'label'] = 0
df_num.loc[df_num.label == "Keylogger", 'label'] = 1
df_num.drop('Class', axis=1, inplace=True)
df_num.sample(5)

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
78885,57464.0,80.0,6.0,42837317.0,3.0,1.0,0.0,0.0,0.0,0.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
92806,55271.0,80.0,6.0,630946.0,3.0,3.0,1448.0,193.0,1448.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
430155,60961.0,53.0,17.0,24814.0,1.0,1.0,34.0,50.0,34.0,34.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
194455,52087.0,80.0,6.0,559741.0,3.0,3.0,646.0,317.0,646.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
224921,59277.0,443.0,6.0,52163068.0,25.0,21.0,1965.0,4558.0,754.0,0.0,...,32.0,638079.0,0.0,638079.0,638079.0,51103460.0,0.0,51103460.0,51103460.0,0


# ** Distribution Classes**

In [10]:
df_num.groupby('label').size()

label
0    308813
1    214782
dtype: int64

# **Remove Useless Features**

In [11]:
for col in (df_num.iloc[:,:-1].columns):
    if(df_num[col].min()==df_num[col].max()):
            df_num.drop(col, axis=1, inplace=True)

# **Feature Scaling**

In [12]:
# Normalization OR Standardization
def standardize(df,col):
    df[col]= (df[col]-df[col].mean())/(df[col].std()) # Standardization
    #df[col]= (df[col]-df[col].min())/(df[col].max()-df[col].min()) #Normalization

for i in (df_num.iloc[:,:-1].columns):
    standardize (df_num,i)

df_num.head()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,-0.210447,-0.368579,-0.468868,0.042412,-0.004811,-0.036722,-0.013233,-0.038629,-0.021233,-0.212203,...,0.004137,-0.183877,-0.063066,-0.175005,-0.173854,-0.290551,-0.121636,-0.296481,-0.277622,0
1,0.828487,-0.368579,-0.468868,-0.465677,0.945480,0.575336,-0.004001,0.556642,0.815442,-0.212203,...,0.004137,-0.183877,-0.063066,-0.175005,-0.173854,-0.290551,-0.121636,-0.296481,-0.277622,0
2,-2.027848,2.839488,-0.468868,-0.483831,-0.076338,-0.055364,-0.018437,-0.046536,-0.435509,-0.212203,...,0.004137,-0.183877,-0.063066,-0.175005,-0.173854,-0.290551,-0.121636,-0.296481,-0.277622,0
3,-0.821057,-0.393449,2.036972,-0.460353,-0.086556,-0.052257,-0.018736,-0.046050,-0.478832,0.522899,...,0.004137,-0.183877,-0.063066,-0.175005,-0.173854,-0.290551,-0.121636,-0.296481,-0.277622,0
4,0.759549,-0.368579,-0.468868,-0.166949,-0.066120,-0.055364,-0.019463,-0.046536,-0.584432,-0.212203,...,0.004135,-0.183877,-0.063066,-0.175005,-0.173854,-0.290551,-0.121636,-0.296481,-0.277622,0


In [13]:
target=['label']
features = [c for c in df_num.columns if c!="label"]

In [14]:
X_new = SelectKBest(f_classif, k=48).fit_transform(df_num[features], df_num[target])
X_new=df_num[features]

/Users/Abbas/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(df_num[features], df_num[target].astype('int'))
modelC = SelectFromModel(lsvc, prefit=True)
X_new = modelC.transform(df_num[features])
X_new.shape

/Users/Abbas/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Abbas/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/Abbas/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


(523595, 53)

# **Split DataSet**

In [16]:
X = df_num[features].values # Features
X = df_num[features[0:18]].values
X=X_new
y = df_num[target].values # Target

X=X.astype(np.float32)
y=y.astype(np.float32)

X.shape,y.shape,

((523595, 53), (523595, 1))

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [18]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((418876, 53), (104719, 53), (418876, 1), (104719, 1))

In [22]:
learning_rate=0.001
batch_size=512
epochs = 25

model_save = ModelCheckpoint('T_Keylogging.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 50, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.75, 
                              patience = 10, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

In [23]:
def Create_Model_ANN(num_columns, num_labels,learning_rate):
    model = Sequential()
    model.add(Dense(64,input_dim=num_columns, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_labels,activation='sigmoid'))
    
    adam = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    #adam = Adam(lr=learning_rate)
    
    model.compile(loss='binary_crossentropy', optimizer=adam,metrics=[keras.metrics.AUC(name='auc')])
    
    return model

model = Create_Model_ANN(X_test.shape[1], 1,learning_rate)

In [25]:
from tensorflow import keras
model.load_weights('/Users/Abbas/Desktop/college/research work/dr_sheikh_abdullah/T_Keylogging.h5')

OSError: Unable to open file (file signature not found)

# **Build ANN Model**

# **Transfer learning**

In [26]:
X = df_num[features].values # using different input size(50)
y = df_num[target].values # Target

X=X.astype(np.float32)
y=y.astype(np.float32)

X.shape,y.shape,

((523595, 67), (523595, 1))

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [28]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((418876, 67), (104719, 67), (418876, 1), (104719, 1))

# ** Model**

In [29]:
def Create_Model_ANN(num_columns, num_labels,learning_rate):
    model = Sequential()
    model.add(Dense(64,input_dim=num_columns, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_labels,activation='sigmoid'))
    
    adam = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    #adam = Adam(lr=learning_rate)
    
    model.compile(loss='binary_crossentropy', optimizer=adam,metrics=[keras.metrics.AUC(name='auc')])
    
    return model

model = Create_Model_ANN(X_test.shape[1], 1,learning_rate)

/Users/Abbas/anaconda3/lib/python3.11/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [30]:
input_size=X_train.shape[1] # 50

model_new = keras.Sequential()
# Add input layer 50 

w0=model.layers[0].get_weights()[0][:,:]
b0=model.layers[0].get_weights()[1]

model_new.add(Dense(64,input_dim=input_size, activation='relu'))
model_new.layers[0].set_weights([w0, b0])

# Add trained layers (with weight)
for layer in model.layers[1:7]:
    model_new.add(layer)
    
# add output layer  sigmoid or softmax..    
model_new.add(Dense(1,activation='sigmoid'))
#model_new.layers[0].set_weights([w0, b0])

adam = Adam(lr=learning_rate)
model_new.compile(loss='binary_crossentropy', optimizer=adam,metrics=[keras.metrics.AUC(name='auc')])

In [31]:
print(model_new.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 64)                4352      
                                                                 
 dense_17 (Dense)            (None, 128)               8320      
                                                                 
 dense_18 (Dense)            (None, 256)               33024     
                                                                 
 dense_19 (Dense)            (None, 512)               131584    
                                                                 
 dense_20 (Dense)            (None, 256)               131328    
                                                                 
 dense_21 (Dense)            (None, 128)               32896     
                                                                 
 dense_22 (Dense)            (None, 64)               

In [32]:
history_new=model_new.fit(X_train,
          y_train,
          validation_data=(X_test,y_test),
          callbacks = [model_save, early_stop, reduce_lr],
          verbose=1,
          batch_size=batch_size,
          epochs=epochs)

Epoch 1/25


: 

# **Evaluation**

In [ ]:
y_pred = model_new.predict(X_test)
AUC = metrics.roc_auc_score(y_test,y_pred)
print("AUC: {:.3f}".format(AUC))


NameError: name 'model_new' is not defined

In [ ]:
hist_df1 = pd.DataFrame(history_new.history)
hist_df1.to_csv('history_new.csv')

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(history_new.epoch[-1]+1),history_new.history['val_auc'],label='val_auc')
plt.plot(range(history_new.epoch[-1]+1),history_new.history['auc'],label='auc')
plt.title('auc'); plt.xlabel('Epoch'); plt.ylabel('auc');plt.legend(); 
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(history_new.epoch[-1]+1),history_new.history['val_loss'],label='Val_loss')
plt.plot(range(history_new.epoch[-1]+1),history_new.history['loss'],label='loss')
plt.title('loss'); plt.xlabel('Epoch'); plt.ylabel('loss');plt.legend(); 
plt.show()

In [ ]:
def generate_results(y_test, y_score):
    # print(y_score)
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()
    
generate_results(y_test, y_pred)